In [0]:
#checking the contents of the local file system
dbutils.fs.ls("/FileStore/tables/")

In [0]:
#introducing reusability
#defining a reusable variable
fileroot = "pharma"

#copy file from DBFS to tmp folder
dbutils.fs.cp("/FileStore/tables/" + fileroot + ".zip", "file:/tmp/")

In [0]:
#making the variable accessible to the command line
import os
os.environ['fileroot'] = fileroot

In [0]:
#checking the contents of the tmp directory for the fileroot
#NB fileroot assigns to itself each file name to be prepared

In [0]:
%sh

ls /tmp/

In [0]:
#using fileroot in the command line to unzip the file assigned to it

In [0]:
%sh

unzip -d /tmp /tmp/$fileroot.zip

In [0]:
#checking the contents of the tmp directory for the fileroot

In [0]:
%sh

ls /tmp/

In [0]:
#making a directory in DBFS
#dbutils.fs.mkdirs("/FileStore/tables/"+fileroot+".csv")

In [0]:
#moving the unzipped file to DBFS
dbutils.fs.mv("file:/tmp/"+fileroot+".csv","/FileStore/tables/",True)

In [0]:
#checking the contents of the local file system for the unzipped file
dbutils.fs.ls("/FileStore/tables/"+fileroot+".csv")

In [0]:
#introducing reusability
fileroot = "clinicaltrial_2021"

In [0]:
#introducing reusability
clinicaltrial_file = ("/FileStore/tables/"+fileroot+".csv")
pharma = ('dbfs:/FileStore/tables/pharma.csv')

In [0]:
#checking the contents of the folder
dbutils.fs.ls("/FileStore/tables/"+fileroot+".csv")

In [0]:
#using dbutils.fs to view the first 65536 bytes of the local file /FileStore/tables/"+fileroot+".csv"

dbutils.fs.head("/FileStore/tables/"+fileroot+".csv")

In [0]:
#reading each line of the clinicaltrial_2021 csv file as one element of clinicaltrial_rdd

clinicaltrial_rdd = sc.textFile(clinicaltrial_file)

#filtering out the header
header = clinicaltrial_rdd.first()
clinicaltrial_rdd = clinicaltrial_rdd.filter(lambda k:k !=header)

In [0]:
#retrieving all elements of the dataset (from all nodes) to the driver node

clinicaltrial_rdd.collect()

In [0]:
#The number of studies in the dataset.

clinicaltrial_rdd.distinct().count()

In [0]:
#Returning the count of each unique value in this RDD as a dictionary of (value, count) pairs.

clinicaltrial_rdd.countByValue()

In [0]:
# list all the types (as contained in the Type column) of studies in the dataset along with the frequencies of each type.

In [0]:
#mapping and splitting by delimiter
clinicaltrial_rdd_st = clinicaltrial_rdd.map(lambda k:(k.split("|") [5],1))
clinicaltrial_rdd_st.take(10)

In [0]:
#adding by key and sorting ascendingly
clinicaltrial_rdd_studies = clinicaltrial_rdd_st.reduceByKey(lambda a,b: a+b)\
.sortBy(lambda k: k[1], False)

clinicaltrial_rdd_studies.take(4)

In [0]:
#number of clinicaltrial studies 
clinicaltrial_rdd_studies.count()

In [0]:
# top 5 conditions (from Conditions) with their frequencies.

In [0]:
#selecting index, splitting by delimiters,mapping, and flatMapping
clinicaltrial_rdd_cd = clinicaltrial_rdd.map(lambda k:(k.split("|") [7]))\
.filter(lambda k:k !='')\
.flatMap(lambda k:k.split(","))\
.map(lambda k:(k.strip(), 1))\


clinicaltrial_rdd_cd.take(5)

In [0]:
#adding by key and sorting ascendingly
clinicaltrial_rdd_condition = clinicaltrial_rdd_cd.reduceByKey(lambda j,f: j+f)\
.sortBy(lambda k: k[1], False)

clinicaltrial_rdd_condition.take(5)

In [0]:
#Find the 10 most common sponsors that are not pharmaceutical companies, alongwith the number of clinical trials they have sponsored. 

In [0]:
#Load the pharma file into RDD
pharma_rdd = (sc.textFile(pharma)
              .map(lambda x: x.replace('"', ''))
              .map(lambda line: line.split(','))
              )

# Print the first three lines of data
pharma_rdd.take(2)

In [0]:
pharma_rdd = sc.textFile(pharma)
header_ii = pharma_rdd.first()
pharma_rdd = pharma_rdd.filter(lambda k:k !=header_ii)

pharma_rdd.take(3)

In [0]:
pharma_rdd.collect()

In [0]:
#splitting by delimiter, replacing, selecting index, and mapping
pcc_rdd = pharma_rdd.map(lambda k: k.split(","))\
.map(lambda k: (k[1].replace('"', '')))\
.map(lambda k: (k,1)) 

pcc_rdd.take(3)

In [0]:
#splitting by delimiter, replacing, selecting index, and mapping
header = clinicaltrial_rdd.first()
clinicaltrialsponsors_rdd = clinicaltrial_rdd.filter(lambda k:k !=header)\
.map(lambda k: k.split("|"))\
.map(lambda k: (k[1],1))

clinicaltrialsponsors_rdd.take(3)

In [0]:
#joining sponsors data with parent companies that are not pharmaceutical companies
sponsors = clinicaltrialsponsors_rdd.leftOuterJoin(pcc_rdd)\
.filter(lambda k: k[1][1]==None)\
.map(lambda k: (k[0],1))\
.reduceByKey(lambda a,b: a+b)\
.sortBy(lambda k: k[1], False)

sponsors.take(10)

In [0]:
#Plot number of completed studies each month in a given year – for the submission dataset, the year is 2021. You need to include your visualization as well as a table of all the values you have plotted for each month

In [0]:
header = clinicaltrial_rdd.first()
completedstudies_rdd = clinicaltrial_rdd.filter(lambda k:k !=header)\
.map(lambda s:s.split("|"))\
.filter(lambda s:s[2]=="Completed")\
.map(lambda s:s[4].split(" "))\
.filter(lambda s:s[0] !="")\
.filter(lambda s:s[1] =="2021")\
.map(lambda s:(s[0],1))\
.reduceByKey(lambda j,k: j+k)\
.sortBy(lambda k:k[0])

completedstudies_rdd.collect()

In [0]:
import calendar
month_sort = {i:e for e,i in enumerate(calendar.month_abbr[1:],1)}
scs = completedstudies_rdd.sortBy(keyfunc=lambda v: month_sort.get(v[0]))
scs.collect()

In [0]:
#converting RDD to DF
cs_dff =sc.textFile("scs")
cs_dff = completedstudies_rdd.toDF()
cs_dff.show()

In [0]:
cs_dff = cs_dff.withColumnRenamed("_1","Month" ). \
withColumnRenamed("_2","No_of_completed_studies")

cs_dff.show()

In [0]:
cs_dff= cs_dff.withColumn("Month",cs_dff.Month.cast("date"))

In [0]:
cs_dff.printSchema()


In [0]:
#cs_dff.write.csv("/FileStore/tables/cs_.csv")

In [0]:
%sql
DROP TABLE IF EXISTS cs_;
CREATE TABLE cs_
USING csv
OPTIONS (path "dbfs:/FileStore/tables/cs_.csv", header = "False" ,inferSchema = "True")

In [0]:
%sql
SHOW TABLES

In [0]:
%sql
SELECT * 
FROM cs_;